In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import transforms, datasets, utils
from torch.autograd import Variable

torch.manual_seed(42)

# Step 1 Build data pipeline and import image dataset (split into training set and test set 5:5)

In [2]:
# data_dir is the address where you store your data data, I put it here under relative path
data_dir = "D:/zhuomian/2022_project/data"
split = 0.5
batch_size = 32
learning_rate = 0.001
num_epochs = 30

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("device:", device)

device: cpu


In [3]:
# Build data pre-processing
data_transform = transforms.Compose([transforms.ToTensor(),
                                     transforms.Resize((56,56)),
                                     transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])

# Import data into dataset
dataset = datasets.ImageFolder(root = data_dir, transform = data_transform)

# Now divide the data set into a training set and a test set
len_imgs = len(dataset.imgs)
train_set, test_set = torch.utils.data.random_split(dataset, [int(len_imgs*split), len_imgs-int(len_imgs*split)]) 

# Final construction of the data pipeline
train_loader = torch.utils.data.DataLoader(train_set,batch_size = batch_size,shuffle = True)
test_loader = torch.utils.data.DataLoader(test_set,batch_size = batch_size,shuffle = False)

# Step 2 Build the model Customized loss function and optimizer

In [8]:
class CNN_model(nn.Module):
    def __init__(self):
        super(CNN_model, self).__init__()
        # conv2d Parameters nn.Conv2d(in_channels=3, out_channels=32,kernel_size=5,stride=1,padding="same")
        # MaxPool2d Parameters nn.MaxPool2d(kernel_size=2, stride=2)
        
        self.conv00 = nn.Sequential(         
            nn.Conv2d(3, 16, 1, 1, padding = "same"),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.Conv2d(16, 16, 3, 1, padding = "same"),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.Conv2d(16, 3, 1, 1, padding = "same"),
            nn.BatchNorm2d(3),
        )
        self.conv0 = nn.Sequential(         
            nn.Conv2d(64, 96, 1, 1, padding = "same"),
            nn.BatchNorm2d(96),
            nn.ReLU(),
            nn.Conv2d(96, 96, 3, 1, padding = "same"),
            nn.BatchNorm2d(96),
            nn.ReLU(),
            nn.Conv2d(96, 64, 1, 1, padding = "same"),
            nn.BatchNorm2d(64),
            nn.ReLU(),
        )
        
        self.conv1 = nn.Sequential(         
            nn.Conv2d(3, 32, 3, 1, padding = "same"),     
            nn.ReLU(),                      
            nn.MaxPool2d(2, 2) 
        )
        
        self.conv2 = nn.Sequential(         
            nn.Conv2d(32, 64, 3, 1, padding = "same"),     
            nn.ReLU(),                      
            nn.MaxPool2d(2, 2)                
        )
        
        self.conv3 = nn.Sequential(         
            nn.Conv2d(64, 32, 3, 1, padding = "same"),     
            nn.ReLU(),                      
            nn.MaxPool2d(2, 2)                
        )
        
        # Fully connected layer with 21 output categories
        self.fc1 = nn.Sequential(nn.Linear(32 * 7 * 7, 128),nn.ReLU())
        self.fc2 = nn.Sequential(nn.Linear(128, 21))
    def forward(self, x):
        identity = x
        x = self.conv00(x)
        x = identity + x
        x = nn.ReLU()(x)
        x = self.conv1(x)
        x = self.conv2(x)
        identity = x
        x = self.conv0(x)
        x = identity + x
        x = self.conv3(x)
        # Flattening treatment x.view
        x = x.view(x.size(0), -1)       
        x = self.fc1(x)
        output = self.fc2(x)
        return output    # return x for visualization

In [9]:
model = CNN_model()
model.to(device)
# Choose whether to print the model or not, here it will not be printed, just comment
print(model)

# Select loss function and optimizer
loss_func = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),
                       lr=learning_rate, betas=(0.9, 0.999),
                       eps=1e-07, amsgrad=False)

CNN_model(
  (conv00): Sequential(
    (0): Conv2d(3, 16, kernel_size=(1, 1), stride=(1, 1), padding=same)
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (4): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): Conv2d(16, 3, kernel_size=(1, 1), stride=(1, 1), padding=same)
    (7): BatchNorm2d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv0): Sequential(
    (0): Conv2d(64, 96, kernel_size=(1, 1), stride=(1, 1), padding=same)
    (1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Conv2d(96, 96, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (4): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): Conv2d(96, 64, kernel_size=(1, 1), stride=(1, 1), padding=

# Step 3 Pass in the model to start training and testing

In [10]:
# Train the model and record the accuracy and loss values for each epoch during training
def train(num_epochs, model, loaders):
    model.train()   
    
    # The first stores the correct rate and the second stores the loss value
    Accy_list = []
    Loss_list = []
    
    total_step = len(loaders)    
    for epoch in range(num_epochs):
        correct = 0
        L = 0
        for i, (images, labels) in enumerate(loaders):
            images = images.to(device)
            labels = labels.to(device)              
            outputs = model(images)
            loss = loss_func(outputs, labels)
            
            # Clear the previous gradient 
            optimizer.zero_grad()           
            
            # Backpropagate and update the parameters
            loss.backward()               
            optimizer.step()                
            
            _, predicted = torch.max(outputs.data, 1)
            correct += (predicted == labels).float().sum()
            L += loss.item()
            if (i+1) % 9 == 0:
                print ('Epoch: [{}/{}]\t|\tStep: [{}/{}]\t|\tLoss: {:.4f}' 
                       .format(epoch + 1, num_epochs, i + 1, total_step, loss.item()))
                
        accuracy = 100 * correct / len(train_set)
        print(accuracy.item())
        print("epoch: {:02d} | Accuracy: {:.2f} Loss: {:.4f}\n"
              .format(epoch + 1, accuracy, L))
        
        # Add the correct and lost values for this epoch to the corresponding list
        Accy_list.append(accuracy.item())
        Loss_list.append(L)
              
    return Accy_list, Loss_list


Accy_list, Loss_list = train(num_epochs, model, train_loader)

Epoch: [1/30]	|	Step: [9/27]	|	Loss: 3.1001
Epoch: [1/30]	|	Step: [18/27]	|	Loss: 3.0405
Epoch: [1/30]	|	Step: [27/27]	|	Loss: 2.9711
6.183115482330322
epoch: 01 | Accuracy: 6.18 Loss: 81.7393

Epoch: [2/30]	|	Step: [9/27]	|	Loss: 2.6562
Epoch: [2/30]	|	Step: [18/27]	|	Loss: 2.6738
Epoch: [2/30]	|	Step: [27/27]	|	Loss: 2.4448
18.192626953125
epoch: 02 | Accuracy: 18.19 Loss: 72.4624

Epoch: [3/30]	|	Step: [9/27]	|	Loss: 1.9709
Epoch: [3/30]	|	Step: [18/27]	|	Loss: 1.8435
Epoch: [3/30]	|	Step: [27/27]	|	Loss: 1.9950
33.1747932434082
epoch: 03 | Accuracy: 33.17 Loss: 59.6997

Epoch: [4/30]	|	Step: [9/27]	|	Loss: 1.4416
Epoch: [4/30]	|	Step: [18/27]	|	Loss: 1.6400
Epoch: [4/30]	|	Step: [27/27]	|	Loss: 1.0539
51.60523223876953
epoch: 04 | Accuracy: 51.61 Loss: 44.0792

Epoch: [5/30]	|	Step: [9/27]	|	Loss: 1.1589
Epoch: [5/30]	|	Step: [18/27]	|	Loss: 0.9988
Epoch: [5/30]	|	Step: [27/27]	|	Loss: 0.7274
67.53864288330078
epoch: 05 | Accuracy: 67.54 Loss: 28.8213

Epoch: [6/30]	|	Step: [9/27]	

In [11]:
print(Accy_list)

[6.183115482330322, 18.192626953125, 33.1747932434082, 51.60523223876953, 67.53864288330078, 79.90487670898438, 88.10939025878906, 89.53627014160156, 92.1521987915039, 95.00594329833984, 96.19500732421875, 94.76813507080078, 97.97859954833984, 99.40547180175781, 99.88109588623047, 99.52437591552734, 98.4542236328125, 98.8109359741211, 99.04875183105469, 99.52437591552734, 99.28656005859375, 99.52437591552734, 99.52437591552734, 99.16765594482422, 100.0, 99.28656005859375, 99.52437591552734, 100.0, 99.88109588623047, 100.0]


In [12]:
print(Loss_list)

[81.73928689956665, 72.4624228477478, 59.69972050189972, 44.07915008068085, 28.821273922920227, 17.961145967245102, 10.474426053464413, 7.8083693608641624, 6.436055019497871, 4.775013010948896, 3.8100471906363964, 4.0998710840940475, 2.238430205732584, 1.1688264994882047, 0.41583608044311404, 0.437314854003489, 1.4841654994525015, 0.9778569438494742, 1.0662839419674128, 0.6389986264985055, 0.4633585677947849, 0.46640564512927085, 0.6203633901604917, 0.9008481834316626, 0.22013960499316454, 1.0604166086995974, 0.44238691485952586, 0.1461748022120446, 0.09261186624644324, 0.04241151952010114]


In [13]:
# Test models
def test(model, loaders):
    model.eval()
    with torch.no_grad():
        correct = 0
        L = 0
        for images, labels in loaders:
            images = images.to(device)
            labels = labels.to(device)              
            outputs = model(images)
            # Get the loss value and add it up
            loss = loss_func(outputs, labels)
            L += loss.item()
            
             # Get the correct number of predicted samples
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).float().sum()
            
    accuracy = (100 * correct / len(test_set))
    print("Test data | Accuracy: {:.2f} %, Loss: {:.4f} ".format(accuracy, L))
    
    return accuracy, L

Test_acc, Test_loss = test(model, test_loader)

Test data | Accuracy: 82.52 %, Loss: 29.7056 
